# METİN TANIMA UYGULAMASI

## Geliştirme ortamına atanan ekran kartının kontrolü

* Geliştirmeye başlamadan önce geliştirme ortamında GPU kullanımının aktifleştirildiğinden emin olunması için yukarıda "Runtime" sekmesinden "Change runtime type"a tıklayalım. Eğer GPU seçeneği seçili değilse seçim listesinden GPUyu seçelim. Bu durumda bu Python Notebook'u tekrardan yüklememiz gerekebilir.
* GPU'nun aktif olup olmadığını ve bize atanan GPU'nun özelliklerini aşağıdaki komutu çalıştırarak görebiliriz.

In [ ]:
!nvidia-smi

## Colab ile Google Drive bağlantısının kurulması

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

In [ ]:
# Kullanılacak diğer kütüphanelerin yüklenmesi
import os
import torch
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = (15.0, 12.0) # görselin boyutları
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

## Google Drive'da çalışılacak klasörün oluşturulması ve klasörün konumuna geçiş

In [ ]:
%cd /content/gdrive/MyDrive/Calisma_ortami/Bilgisayarli_Goru_Uygulama_Alanlari/Optik_Karakter_Tanima/MetinTanimaUygulamasi
!mkdir TextRecognition
%cd TextRecognition

## Github'dan ilgili repository'nin çalışma klasörümüze yüklenmesi ve önceden eğitilmiş modellerin indirilmesi

In [ ]:
# Metin Tanıma repo'sunun kopyalanması
!git clone https://github.com/clovaai/deep-text-recognition-benchmark.git
# Kodun bulunduğu konuma giriş
%cd deep-text-recognition-benchmark
# Modellerin koyulacağı klasörün oluşturulması
!mkdir pths
# Model klasörüne giriş
%cd pths
# Önceden eğitilmiş modellerin indirilmesi
!gdown https://drive.google.com/uc?id=1b59rXuGGmKne1AuHnkgDzoYgKeETNMv9
#!gdown https://drive.google.com/uc?id=1ajONZOgiG9pEYsQ-eBmgkVbMDuHgPCaY
# Kodun bulunduğu klasöre geri dönüş
%cd /content/gdrive/MyDrive/Calisma_ortami/Bilgisayarli_Goru_Uygulama_Alanlari/Optik_Karakter_Tanima/MetinTanimaUygulamasi/TextRecognition/deep-text-recognition-benchmark

## Test resimlerinden metinlerin tanınması

In [ ]:
plt.rcParams['figure.figsize'] = (10.0, 8.0) # görselin boyutları
resim_klasoru = '/content/gdrive/MyDrive/Calisma_ortami/Bilgisayarli_Goru_Uygulama_Alanlari/Optik_Karakter_Tanima/MetinTanimaUygulamasi/TextRecognition/deep-text-recognition-benchmark/demo_image'
resim_dosyalari = os.listdir(resim_klasoru)
f, axarr = plt.subplots(5,2)
f.tight_layout()
for i in range(5):
    for j in range(2):
        resim_dosyasi = os.path.join(resim_klasoru, resim_dosyalari[i*2 + j])
        axarr[i, j].imshow(Image.open(resim_dosyasi))
        axarr[i, j].axis('off')
f.show()
plt.rcParams['figure.figsize'] = (15.0, 12.0) # görselin boyutları

In [ ]:
!python /content/gdrive/MyDrive/Calisma_ortami/Bilgisayarli_Goru_Uygulama_Alanlari/Optik_Karakter_Tanima/MetinTanimaUygulamasi/TextRecognition/deep-text-recognition-benchmark/demo.py \
--Transformation TPS --FeatureExtraction ResNet --SequenceModeling BiLSTM --Prediction Attn \
--image_folder /content/gdrive/MyDrive/Calisma_ortami/Bilgisayarli_Goru_Uygulama_Alanlari/Optik_Karakter_Tanima/MetinTanimaUygulamasi/TextRecognition/deep-text-recognition-benchmark/demo_image/ \
--saved_model /content/gdrive/MyDrive/Calisma_ortami/Bilgisayarli_Goru_Uygulama_Alanlari/Optik_Karakter_Tanima/MetinTanimaUygulamasi/TextRecognition/deep-text-recognition-benchmark/pths/TPS-ResNet-BiLSTM-Attn.pth

# Metin Tanıma detaylı anlatım

In [ ]:
# resim ve model dosyalarının konumları
resim_dosyası = '/content/gdrive/MyDrive/Calisma_ortami/Bilgisayarli_Goru_Uygulama_Alanlari/Optik_Karakter_Tanima/MetinTanimaUygulamasi/TextRecognition/deep-text-recognition-benchmark/demo_image/demo_1.png'
model_dosyası = '/content/gdrive/MyDrive/Calisma_ortami/Bilgisayarli_Goru_Uygulama_Alanlari/Optik_Karakter_Tanima/MetinTanimaUygulamasi/TextRecognition/deep-text-recognition-benchmark/pths/TPS-ResNet-BiLSTM-Attn.pth'

# resmin yüklenmesi
resim = Image.open(resim_dosyası).convert('L')
plt.imshow(resim)
plt.show()

In [ ]:
%cd /content/gdrive/MyDrive/Calisma_ortami/Bilgisayarli_Goru_Uygulama_Alanlari/Optik_Karakter_Tanima/MetinTanimaUygulamasi/TextRecognition/deep-text-recognition-benchmark/

In [ ]:
from model import Model
from utils import AttnLabelConverter

class ModelParametreleri:

    def __init__(self):
        self.character = '0123456789abcdefghijklmnopqrstuvwxyz'
        self.Transformation = 'TPS'
        self.FeatureExtraction = 'ResNet'
        self.SequenceModeling = 'BiLSTM'
        self.Prediction = 'Attn'
        self.num_fiducial = 20
        self.output_channel = 512
        self.hidden_size = 256
        self.imgH = 32  # modele verilecek resmin boyu
        self.imgW = 100  # modele verilecek resmin eni
        self.input_channel = 1  # model siyah beyaz resimler üzerinde çalışıyor
        self.num_class = len(self.character) + 2  # buradaki +2, tahmine başlangıç ve bitiş iki özel karakteri için ekleniyor
        self.batch_max_length = 25

# modelin oluşturukması için gerekli parametrelerin oluşturulması
opt = ModelParametreleri()

# tahminin maksimum karakter sayısı ve metnin tahmininin tutulacağı değişkenler
tahminin_uzunlugu = torch.cuda.IntTensor([opt.batch_max_length])
tahminin_metni = torch.cuda.LongTensor(1, opt.batch_max_length + 1).fill_(0)

# modelin tahminlerini metne çeviren modülün oluşturulması
converter = AttnLabelConverter(opt.character)
# modelin oluşturulması ve önceden eğitilmiş modelin yüklenmesi
model = Model(opt)
# torch.nn.DataParallel ile eğitilen modeller yine bu şekilde çağrılmalıdır
model = torch.nn.DataParallel(model).cuda()
model.load_state_dict(torch.load(model_dosyası))

In [ ]:
from dataset import ResizeNormalize

# resmin, modele verebilmek için yeniden boyutlandırılması (100x32)
# resmin oytorch tensörüne dönüştürülmesi
# resmin normalize edilmesi
donusturucu = ResizeNormalize((opt.imgW, opt.imgH))

# resme gerekli işlemleri uygulayıp GPU'ya taşıma
resim_tensoru = donusturucu(resim).cuda()
resim_tensoru = resim_tensoru.view(1, *resim_tensoru.size())

In [ ]:
# tahminin yapılması
tahmin = model(resim_tensoru, tahminin_metni, is_train=False)

# en yüksek ihtimalle tahmin edilen karakterlerin indekslerini çıkarır
_, tahmin_indeksi = tahmin.max(2)
# buradaki sıfır resim sayımız sadece 1 olduğundan ilk elemanı almak için kullanılıyor
tahmin_dizisi = converter.decode(tahmin_indeksi, tahminin_uzunlugu)[0]
 # "bitiş" özel karakterinin "([s])" çıkarılması
tahmin_dizisi = tahmin_dizisi[:tahmin_dizisi.find('[s]')]
# tahmin edilen metnin resim ile gösterilmesi
print(tahmin_dizisi)
plt.imshow(resim)
plt.show()

# Bu Python Notebook üzerinde her bir çıktının ne olduğunu print() metodu ile yazdırarak, modelin çalışma şeklini daha detaylı anlayabilirsiniz.